In [ ]:
import pandas as pd
import tqdm
import os
from torch.utils.tensorboard import SummaryWriter

df_train = pd.read_csv('data_le.txt', sep=r"\s+", header=None, names=['x1', 'x2', 'y'])
df_test = pd.read_csv('data_cc1.txt', sep=r"\s+", header=None, names=['x1', 'x2', 'y'])

df_train['y'] = (df_train['y'] + 1) / 2
df_test['y'] = (df_test['y'] + 1) / 2

In [ ]:
from nn.dataset import DataSet
from nn.model import Sequential
from nn.linear import Linear
from nn.activation import Sigmoid
from nn.loss import BCE
from nn.optimizer import SGD
from nn.weights import xavier_normal

In [ ]:
train_dataset = DataSet(df_train[['x1', 'x2']].values, df_train['y'].values)
test_dataset = DataSet(df_test[['x1', 'x2']].values, df_test['y'].values, batch_size=300)

In [ ]:
from sklearn import metrics as sk_metrics

def calculate_metrics(pred, target, thresh=0.5):
    metrics = {}
    metrics['ap'] = sk_metrics.average_precision_score(target, pred)
    try:
        metrics['auc'] = sk_metrics.roc_auc_score(target, pred)
    except:
        metrics['auc'] = metrics['ap']
    metrics['acc'] = sk_metrics.accuracy_score(target, pred > thresh)
    metrics['p'] = sk_metrics.precision_score(target, pred > thresh)
    metrics['r'] = sk_metrics.recall_score(target, pred > thresh)
    metrics['f1'] = sk_metrics.f1_score(target, pred > thresh)
    return metrics

In [ ]:
params = {
    'batch_size': 32,
    'epoches': 100,
    'lr': 1e-1,
}

In [ ]:
model = Sequential(
    layers=[       
        Linear(2, 1, weights_init=xavier_normal),
        Sigmoid()
    ],
    opt=SGD(lr=params['lr']))
loss_fn = BCE()

In [ ]:
exp_name = "test"
i = 1
fold = f'demo_1'
folder = f'./{fold}/{exp_name}'
while os.path.isdir(folder):
    i += 1
    folder = f'./{fold}/{exp_name}_{i}'
writer = SummaryWriter(folder)

for epoch in range(params['epoches']):
    # Train
    with tqdm.tqdm(train_dataset, unit="batches") as tepoch:
        epoch_metrics = {}
        counter = 0
        for data, target in tepoch:
            tepoch.set_description(f"Epoch {epoch} / Train")
            
            output = model(data).squeeze()
            loss = loss_fn(output, target)

            # Metrics
            metrics = calculate_metrics(output, target)
            metrics['loss'] = loss

            # Tensorboard
            cur_batch = len(target)
            if counter:
                for k, v in metrics.items():
                    epoch_metrics[k] = (epoch_metrics[k]*counter + v*cur_batch)/(counter + cur_batch)
            else:
                epoch_metrics = metrics.copy()
            counter += cur_batch

            # Backward
            model.backward(loss_fn.backward())

            tepoch.set_postfix(**epoch_metrics)
        for k, v in epoch_metrics.items():
            writer.add_scalar(f"Train/{k}", v, global_step=epoch)

    # Test
    with tqdm.tqdm(test_dataset, unit="batch") as tepoch:
        for data, target in tepoch:
            tepoch.set_description(f"Epoch {epoch} / Test")
            
            output = model(data).squeeze()
            loss = loss_fn(output, target)

            # Metrics
            metrics = calculate_metrics(output, target)
            metrics['loss'] = loss

            # Tensorboard
            for k, v in metrics.items():
                writer.add_scalar(f"Test/{k}", v, global_step=epoch)

            tepoch.set_postfix(**metrics)